In [ ]:
import yfinance as yf
import pandas as pd
from ativos_precos import ativos_config

In [ ]:
# Ativos
ativos = ativos_config

# Baixando os dados
data = yf.download(ativos, period="max")

# Selecionando apenas as colunas necessárias
data = data['Close']

# Formatar o índice (data) para o formato desejado
data.index = data.index.strftime('%d-%m-%Y')

# Usando a função melt para transformar o DataFrame
df_melted = data.melt(ignore_index=False, var_name='ticker', value_name='valor_cotação')

# Renomeando o índice para 'date'
df_melted.index.name = 'date'

# Resetando o índice para tornar 'date' uma coluna
df_melted = df_melted.reset_index()

# Reorganizando as colunas na ordem desejada
df_organizado = df_melted[['date', 'ticker', 'valor_cotação']]

# Salvar o DataFrame em um arquivo Excel
#df_organizado.to_excel('dados_organizados.xlsx', index=False)

df_organizado.head()

In [ ]:
def baixar_salvar_dividendos(ativos, nome_arquivo_excel='dividendos.xlsx'):
    """
    Baixa os dados de dividendos para uma lista de ativos e salva em um DataFrame
    e em um arquivo Excel.

    Args:
        ativos (list): Uma lista de tickers de ativos (ex: ["HGRU11.SA", "MXRF11.SA"]).
        nome_arquivo_excel (str, optional): O nome do arquivo Excel para salvar os dados.
                                             Padrão é 'dividendos.xlsx'.

    Returns:
        pd.DataFrame: Um DataFrame contendo o histórico de dividendos para todos os ativos.
                      O DataFrame terá colunas 'ticker', 'date' e 'dividendo'.
    """
    todos_dividendos = []
    for ticker in ativos:
        try:
            data_dividends = yf.Ticker(ticker).dividends
            if not data_dividends.empty:
                df_dividends = data_dividends.to_frame(name='dividendo')
                df_dividends['ticker'] = ticker
                df_dividends.index.name = 'date'
                df_dividends = df_dividends.reset_index()
                todos_dividendos.append(df_dividends)
            else:
                print(f"Não foram encontrados dividendos para {ticker}.")
        except Exception as e:
            print(f"Erro ao baixar dividendos para {ticker}: {e}")

    if todos_dividendos:
        df_final_dividendos = pd.concat(todos_dividendos, ignore_index=True)
        try:
            df_final_dividendos['date'] = df_final_dividendos['date'].dt.strftime('%Y-%m-%d')
            df_final_dividendos.to_excel(nome_arquivo_excel, index=False)
            print(f"Dados de dividendos salvos com sucesso em '{nome_arquivo_excel}'.")
            return df_final_dividendos
        except Exception as e:
            print(f"Erro ao salvar os dados de dividendos em Excel: {e}")
            return df_final_dividendos
    else:
        print("Nenhum dado de dividendo foi baixado.")
        return pd.DataFrame(columns=['ticker', 'date', 'dividendo'])

df_dividendos_baixados = baixar_salvar_dividendos(ativos_config, 'historico_dividendos.xlsx')
if df_dividendos_baixados is not None:
     print(df_dividendos_baixados.head())